# Generate some simple geometry for a tube

As a surgical simulator we want to travel the length of the tube and create images from a camera point.

In [108]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib qt

Start by creating a line in 3D.

In [125]:
stepSize = 0.5
smoothness = 0.6
length = 20
endPoint = np.asarray([0,0,0])
centerLine = [endPoint]
randomDir = np.asarray([0,0,1])
for i in range(length):
    randomDir = randomDir + (np.random.rand(3)*2 -
                             np.asarray([smoothness, smoothness, smoothness]))
    randomDir = randomDir / np.linalg.norm(randomDir)
    endPoint = endPoint + (stepSize * randomDir)
    centerLine.append(endPoint)
centerLine

[array([0, 0, 0]),
 array([ 0.10026598, -0.00444437,  0.48982342]),
 array([-0.19567573, -0.0686605 ,  0.8876862 ]),
 array([-0.17938242, -0.51010369,  1.12191353]),
 array([-0.24641104, -0.79382806,  1.52812486]),
 array([-0.2081432 , -1.13831756,  1.88848933]),
 array([-0.29784532, -1.51501909,  2.20479534]),
 array([-0.20398298, -1.56201181,  2.69365273]),
 array([ 0.15086487, -1.54619933,  3.04555183]),
 array([ 0.4999804 , -1.68098335,  3.37714134]),
 array([ 0.6563941 , -1.75761925,  3.84582215]),
 array([ 0.73619335, -1.69375293,  4.33526384]),
 array([ 0.78821666, -1.54345107,  4.80929224]),
 array([ 0.68323593, -1.56189496,  5.29779902]),
 array([ 0.72961019, -1.38592734,  5.76350797]),
 array([ 0.87106604, -1.26092419,  6.22650306]),
 array([ 0.87084583, -1.16776399,  6.71774753]),
 array([ 0.96924314, -0.99987681,  7.17832528]),
 array([ 1.35401231, -0.79854646,  7.4261534 ]),
 array([ 1.59059295, -0.47272083,  7.72257752]),
 array([ 1.64975204e+00, -7.60758723e-03,  7.89627

In [126]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
x = [a[0] for a in centerLine]
y = [a[1] for a in centerLine]
z = [a[2] for a in centerLine]
ax.plot(x,y,z)
plt.show()

Compute two vectors that are orthogonal to the centerline at each point. Those two vectors t and b will be in the plane that contains our circle.

In [129]:
radius = 0.3
circlePoints = 10
circles = []
for i in range(len(centerLine)-1):
    v = centerLine[i+1] - centerLine[i]
    v = v / np.linalg.norm(v)
    r = np.random.rand(3)-0.5
    r = r / np.linalg.norm(r)
    t = np.cross(v, r)
    t = t / np.linalg.norm(t)
    b=t
    # now use these vectors to place our circle
    circle = []
    for j in range(circlePoints):
        angleOfRotation = j*(2*math.pi/circlePoints)
        b2 = b*np.cos(angleOfRotation) + (np.cross(v,b)*np.sin(angleOfRotation) + v*(np.dot(v,b)*(1-np.cos(angleOfRotation))))
        b2 = b2 / np.linalg.norm(b2)
        b2 = radius * b2
        circle.append(centerLine[i+1] + b2)
    circle.append(circle[0])
    circles.append(circle)

We need to put the c

In [131]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
x = [a[0] for a in centerLine]
y = [a[1] for a in centerLine]
z = [a[2] for a in centerLine]
for i in range(len(circles)):
    circle = circles[i]
    xc = [a[0] for a in circle]
    yc = [a[1] for a in circle]
    zc = [a[2] for a in circle]
    ax.plot(xc, yc, zc)
ax.plot(x, y, z)
plt.set_aspect()
plt.show()


AttributeError: module 'matplotlib.pyplot' has no attribute 'set_aspect'